In [61]:
import os
import ee
import geojson
import rasterio
import rioxarray
import xarray as xr
import datetime as dt
from funcs_gee import *

# ----------------------------------------------------------------------

# Define the time range.
start_date = '2022-01-01'
end_date = '2022-12-31'
fn=f's0_{start_date}_{end_date}'
geojson_path='/home/martina/Documents/thesis-codes/WATERSTEM/Pascoli_UNIFI_shape/SanLorenzo2.geojson'
geojson_fd, geojson_fn = os.path.split(geojson_path)
geometry_title, ext = os.path.splitext(geojson_fn)

folder_maps='./Maps/'+geometry_title+'/s0/'
if not os.path.exists(folder_maps):
    os.makedirs(folder_maps)


with open(geojson_path) as f:
    gj = geojson.load(f)

nfeatures = len(gj['features'])
coords_geojson = [gj['features'][i]['geometry']['coordinates'] for i in range(nfeatures)]


# ----------------------------------------------------------------------


def get_s1_sr_cld_col(aoi, start_date, end_date, **kwargs):
    # Import and filter s1 SR.
    return (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .sort('system:time_start'))



def extract_data(image:ee.Image):
    """Ausiliary function to extract data from an Image
    
    This function extracts spatial means and std.dev
    via spatial reducers (reduceRegion).
    Optimal implementation is to map this function
    on a whole ImageCollection via .map() and insert the
    return into a ee.FeatureCollection.
    
    Return
    ------
    ee.Feature
    
    """
    # try:
    mean = image.reduceRegion(**{ 
            'reducer': ee.Reducer.mean(),
            'geometry': region,
        })
        
    dev = image.reduceRegion(**{ 
            'reducer': ee.Reducer.stdDev(),
            'geometry': region,
        })
        
    var = image.reduceRegion(**{
            'reducer':ee.Reducer.variance(),
            'geometry': region,
        })

    # getRegion cannot be applied to single image
    # sigma0 = np.array(image.getRegion(region, scale_mod).getInfo())[1:]
    # sigma0[sigma0==None] = np.nan
    
    properties = {
            'Date': image.get('system:time_start'), # only way to get a timestr is an external operation
            'Geometry': geometry_title,
            'VV[lin]': mean.get('VV'),
            'VH[lin]': mean.get('VH'),
            'Angle[°]': mean.get('angle'),
            'VV_var[lin]': var.get('VV'),
            'VH_var[lin]': var.get('VH'),
            'Orb': image.get('relativeOrbitNumber_start'),
            'Pass': image.get('orbitProperties_pass'),
            # 'Pix': sigma0,
        }
    # except: HTTPError, EEException            
            
    return ee.Feature(None, properties)


# ----------------------------------------------------------------------

# Initialize the Earth Engine module.
ee.Initialize()

region = ee.Geometry.MultiPolygon(coords_geojson)

s1=get_s1_sr_cld_col(region, start_date, end_date)

proj=s1.first().select('VV').projection()
scale_mod=ee.Number(proj.nominalScale()).getInfo()
trans_mod=proj.getInfo()['transform']
crs_mod=proj.getInfo()['crs']

s1 = s1.map(lambda image: image.reproject(crs_mod,trans_mod))


def clean_date(date:int):
    return time.strftime('%x %H', time.localtime((date)/1000))

data = ee.FeatureCollection(s1.map(extract_data))
data_out = data.getInfo()
data_out_to_df = [e.get('properties') for e in data_out.get('features')]; data_out_to_df[0]
df = pd.DataFrame.from_dict(data_out_to_df)
df.Date = df.Date.apply(lambda x : pd.to_datetime(clean_date(x)))
df['VV[dB]'] = df['VV[lin]'].apply(lambda x : lin_db(x))
df['VH[dB]'] = df['VH[lin]'].apply(lambda x : lin_db(x))
df['VV_var[dB]'] = df['VV_var[lin]']/df['VV[lin]']*(10/np.log(10))
df['VH_var[dB]'] = df['VH_var[lin]']/df['VH[lin]']*(10/np.log(10))

dftot = df.set_index('Date'); # dftot

In [62]:
# Get pixels in the region with all bands, lon, lat and time
sigma0 = np.array(s1.getRegion(region, scale_mod).getInfo())
sigma0[sigma0==None]=np.nan

In [63]:
# Build dataframe and clean timestamps
df = pd.DataFrame(sigma0[1:], columns=sigma0[0])
df['datetime'] = pd.to_datetime([dt.datetime.fromtimestamp(float(ts)/1000).strftime("%Y-%m-%d") for ts in df['time']])
drop_columns=['time','id']
df.drop(drop_columns, axis=1, inplace=True)

In [64]:
data_vv = np.array(df.VV.values, dtype=float); data_vh = np.array(df.VH.values, dtype=float)
df['CR'] = data_vh/data_vv
df['VV[dB]'] = df['VV'].apply(lambda x : lin_db(float(x)) if pd.notnull(x) else x)
df['VH[dB]'] = df['VH'].apply(lambda x : lin_db(float(x)) if pd.notnull(x) else x)
df['coords']=list(zip(df.longitude, df.latitude))
dff=df.groupby(by=['datetime','coords']).max()
dff=dff.astype('float')

dates=np.unique(df.datetime)
ds=dff.reset_index().drop(columns=['coords']).set_index(['datetime', 'latitude', 'longitude']).to_xarray()

In [65]:
answ=input('Save nc file? [[y]/n]')
if answ=='y' or answ=='':
    ds.to_netcdf(folder_maps+fn+'.nc')

ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

# Set the Coordinate Reference System (CRS) if it is not set
# You need to know the EPSG code of your CRS
# For example, the EPSG code for WGS84 is 'EPSG:4326'
ds.rio.write_crs("EPSG:4326", inplace=True)

daily_opt=input('Do you want daily images with all bands? (produce first 10) [[y]/n]')
if daily_opt=='' or daily_opt=='y':
    for i, d in enumerate(ds.datetime):
        date_name=str(d.dt.strftime("%Y-%m-%d").values)
        data_daily=ds.sel(datetime=d)
        data_daily.rio.to_raster(folder_maps+date_name+".tif")
        # if i==10: break
multi_opt=input('Do you want multilayer images with VV and VH values only and days as bands? [[y]/n]')
if multi_opt=='' or multi_opt=='y':
    for pol in ['VV','VH']:
        data_var=ds[pol]
        data_var.rio.to_raster(folder_maps+fn+pol+".tif")

Save nc file? [[y]/n] 
Do you want daily images with all bands? (produce first 10) [[y]/n] 
Do you want multilayer images with VV and VH values only and days as bands? [[y]/n] 
